# The *Voice Data* Gathering

The DGX-505 has some default settings for its voices: default volume, octave, pan, dual voice etc. The split voice doesn't change.

In order to get this data (for *Scientific Purposes*™) we can simply eavesdrop on the MIDI messages output when the voice changes.

## Data Gathering Setup
First, we need to avoid any superfluous data being emitted (just to save on cleanup).
This means: only keyboard OUT, External Clock ON, touch response OFF.

For initial settings, we need to first have the Initial Send.

In order to capture all the settings, we need to play a note, as the Octave setting just affects the internal mapping of keys to output notes. We also need to have both Main, and Dual voices active to capture this information. 

Then, we scroll through all the voices, pressing the C3 key for each voice. Start with voice 000 and then scroll all the way through.

We use `slurp.py` to get all this data.


In [1]:
%cd -q '..'

In [2]:
import mido

In [43]:
from commons import mido_util, values

In [20]:
from commons.messages import controlstate, wrappers, controls

In [4]:
with open('documents/data/voicedata.txt') as infile:
    messages = list(mido_util.readin_strings(infile))

In [9]:
len(messages)

4487

In [10]:
state = controlstate.MidiControlState()

In [23]:
state.reset_blank()
for m in messages:
    if m.time > 5:
        break
    print(state.feed(m))


GM System ON
XG System ON
MIDI Master Tuning 0
Reverb Type 03(Hall3)
Chorus Type 1(Chorus1)
0 Bank MSB 0
0 Bank LSB 113
0 Program Change [0,113,0] 001 Live! Grand Piano (PIANO)
1 Bank MSB 0
1 Bank LSB 112
1 Program Change [0,112,48] 054 String Ensemble (STRINGS)
2 Bank MSB 0
2 Bank LSB 0
2 Program Change [0,0,105] 411 Banjo (XG WORLD)
0 Voice Volume 110
1 Voice Volume 36
2 Voice Volume 100
0 Voice Pan 64
1 Voice Pan 64
2 Voice Pan 64
0 Voice Reverb Level 20
1 Voice Reverb Level 50
2 Voice Reverb Level 8
0 Voice Chorus Level 0
1 Voice Chorus Level 0
2 Voice Chorus Level 0
0 RPN MSB 0
0 RPN LSB 0
0 Data Entry MSB: Pitch Bend Range 2
1 RPN MSB 0
1 RPN LSB 0
1 Data Entry MSB: Pitch Bend Range 2
2 RPN MSB 0
2 RPN LSB 0
2 Data Entry MSB: Pitch Bend Range 2
0 Pitch Bend 0
1 Pitch Bend 0
2 Pitch Bend 0
0 Release Time 0
1 Release Time 0
2 Release Time 0


Before time 5 we have the initial send.


In [24]:
{m.type for m in messages if m.time > 5}

{'control_change', 'note_on', 'program_change'}

In [34]:
{m.channel for m in messages if m.time > 5}

{0, 1}

In [27]:
{wrappers.Control(m.control) for m in messages if m.type == 'control_change' and m.time > 5}

{<Control.BANK_LSB: 32>,
 <Control.BANK_MSB: 0>,
 <Control.CHORUS: 93>,
 <Control.REVERB: 91>,
 <Control.VOLUME: 7>}

In [41]:
{m.velocity for m in messages if m.type == 'note_on' and m.time > 55}

{0, 80}

The relevant channels are 0 and 1 for main and dual, with the relevant controls being the bank, chorus, reverb, and voice volumes.

In [35]:
import collections

In [36]:
VoiceSetting = collections.namedtuple('VoiceSetting', 'program volume octave reverb chorus')
ComboSetting = collections.namedtuple('ComboSetting', 'main dual')

In [54]:
combos = []
state.reset_blank()
loct = [None, None]
def vget(ch):
    c = state._channels[ch]
    return VoiceSetting(
        program=c[wrappers.MessageType.PROGRAM_CHANGE],
        volume=c[wrappers.Control.VOLUME],
        octave=loct[ch],
        reverb=c[wrappers.Control.REVERB],
        chorus=c[wrappers.Control.CHORUS]    
    )
for m in messages:
    w = state.feed(m)
    if m.type == 'note_on':
        if m.velocity != 0:
            # note on
            note = values.NoteValue(m.note)
            assert note.note is values.Notes.C
            loct[m.channel] = note.octave - 3
        elif m.channel == 1:
            # note off. This is a good time to record data.
            combos.append(ComboSetting(main=vget(0), dual=vget(1)))            
            loct[:] = [None, None]
            


In [75]:

for main, dual in combos:
    print("{0.program} {0.volume} {0.octave} {0.reverb} {0.chorus} {1.program} {1.volume} {1.octave} {1.reverb} {1.chorus}".format(main, dual))

[0,113,0] 001 Live! Grand Piano (PIANO) 110 0 20 0 [0,112,48] 054 String Ensemble (STRINGS) 36 0 50 0
[0,113,0] 001 Live! Grand Piano (PIANO) 110 0 20 0 [0,112,48] 054 String Ensemble (STRINGS) 36 0 50 0
[0,114,0] 002 Live! Warm Grand Piano (PIANO) 127 0 20 0 [0,112,48] 054 String Ensemble (STRINGS) 32 0 40 0
[0,112,0] 003 Grand Piano (PIANO) 100 0 20 0 [0,118,88] 108 SweetHeaven (SYNTH PAD) 70 0 36 0
[0,112,1] 004 Bright Piano (PIANO) 100 0 18 0 [0,112,48] 054 String Ensemble (STRINGS) 40 0 32 0
[0,112,3] 005 Honky-tonk Piano (PIANO) 90 0 18 0 [0,112,1] 004 Bright Piano (PIANO) 70 1 18 0
[0,112,2] 006 MIDI Grand Piano (PIANO) 68 0 20 32 [0,114,4] 009 Cool! Galaxy Electric Piano (E.PIANO) 60 0 30 70
[0,113,2] 007 CP 80 (PIANO) 88 0 20 30 [0,112,2] 006 MIDI Grand Piano (PIANO) 64 0 20 60
[0,112,6] 008 Harpsichord (PIANO) 82 0 26 6 [0,112,48] 054 String Ensemble (STRINGS) 56 0 50 0
[0,114,4] 009 Cool! Galaxy Electric Piano (E.PIANO) 100 0 24 36 [0,0,1] 138 Bright Piano (XG PIANO) 76 0 24

In [58]:
core_combos = combos[1:-2]

In [59]:
len(core_combos)

494

In [61]:
all(v.main.program.number == i for i, v in enumerate(core_combos, 1))


True

In [117]:

def fmtv(v):
    octave = format(v.octave, "3d" if v.octave == 0 else "+3d")
    prog = "{0.number:03d} {0.name}".format(v.program)
    return ("{:<20} {:3d} {:>3} {:3d} {:3d}").format(prog[:20], v.volume, octave, v.reverb, v.chorus)

def fmtc(c):
    return ",  ".join(fmtv(x) for x in c)

print("{:<20} vol oct rev cho,  {:<20} vol oct rev cho".format("Main Voice", "Dual Voice"))
for c in core_combos:
    print(fmtc(c))

Main Voice           vol oct rev cho,  Dual Voice           vol oct rev cho
001 Live! Grand Pian 110   0  20   0,  054 String Ensemble   36   0  50   0
002 Live! Warm Grand 127   0  20   0,  054 String Ensemble   32   0  40   0
003 Grand Piano      100   0  20   0,  108 SweetHeaven       70   0  36   0
004 Bright Piano     100   0  18   0,  054 String Ensemble   40   0  32   0
005 Honky-tonk Piano  90   0  18   0,  004 Bright Piano      70  +1  18   0
006 MIDI Grand Piano  68   0  20  32,  009 Cool! Galaxy Ele  60   0  30  70
007 CP 80             88   0  20  30,  006 MIDI Grand Piano  64   0  20  60
008 Harpsichord       82   0  26   6,  054 String Ensemble   56   0  50   0
009 Cool! Galaxy Ele 100   0  24  36,  138 Bright Piano      76   0  24   0
010 Cool! Suitcase E  96   0  24  36,  009 Cool! Galaxy Ele  58   0  24  40
011 Cool! Electric P  96   0  24  36,  013 DX Modern Electr  54   0  24   0
012 Funky Electric P  74   0  16  20,  003 Grand Piano       70   0  18  70
013 DX Moder

In [95]:
fmtc(core_combos[2])

'003 Grand Piano                      100  +0  20   0,  108 SweetHeaven                       70  +0  36   0'

In [115]:
import csv

In [116]:
with open('commons/messages/tables/voice_defaults.csv', 'wt') as outfile:
    cw = csv.writer(outfile)
    cw.writerow(["Main Voice", "Main Volume", "Main Octave", "Main Reverb Level", "Main Chorus Level", "Dual Voice", "Dual Volume", "Dual Octave", "Dual Reverb Level", "Dual Chorus Level"])
    for c in core_combos:
        cw.writerow([c.main.program.number, c.main.volume, c.main.octave, c.main.reverb, c.main.chorus, c.dual.program.number, c.dual.volume, c.dual.octave, c.dual.reverb, c.dual.chorus])

In [242]:
with open('documents/voice_default_table.md', 'wt') as outfile:
    headers = ["Main Voice", "M Volume", "M Octave", "M Reverb Lv", "M Chorus Lv",
               "Dual Voice", "D Volume", "D Octave", "D Reverb Lv", "D Chorus Lv"]
    aligns = ["<", ">", ">", ">", ">"]*2
    
    item_fmts = [[ x for l in [["{0.number:03d} {0.name}".format(v.program),
                 "{:3d}".format(v.volume),
                 "{0:{1}3d}".format(v.octave, "+" if v.octave else ""),
                 "{:3d}".format(v.reverb),
                 "{:3d}".format(v.chorus)] for v in c] for x in l] for c in core_combos]
    lens = [max(len(s) for s in x) for x in zip(*([headers] + item_fmts))]
    cf, rf = ["|{}|\n".format("|".join("{{{3}:{2}{0}{1}}}".format(a, l, f, g) for a, l in zip(aligns, lens))) for f, g in [["",""],["-","0"]]]
    outfile.write(cf.format(*headers))
    outfile.write(rf.format(":"))
    for c in item_fmts:
        outfile.write(cf.format(*c))
